###사전 설정

In [1]:
!pip install --upgrade pip
!pip install torch torchvision torchaudio
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install scipy pillow transformers

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ygcnyoa2
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ygcnyoa2
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=c227ae167cdc6329d15ef8eb07a9dc090c1b233f2b293ecb56c7be16f0cfdaa6
  Stored in directory: /tmp/pip-ephem-wheel-cache-8vopnaao/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
Using device: cuda


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from  tqdm import tqdm
import random
import torch
import torchvision
import torchvision.transforms as T
from torchvision.transforms import ToPILImage
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
import os
from torchvision import transforms
import scipy.io
import shutil
import clip
from PIL import Image
import pandas as pd

##wikiart 전처리

###데이터셋 다운로드

In [ ]:
import kagglehub

path = kagglehub.dataset_download("steubk/wikiart")

print("Path to dataset files:", path)

In [ ]:
!mv "{path}" /content/

In [ ]:
#데이터셋 분리 코드
csv_path = "/content/dataset/classes.csv"
data = pd.read_csv(csv_path)

source_folder = "/content/dataset"

train_folder = "/content/dataset/train"
test_folder = "/content/dataset/test"
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)


def extract_class_and_filename(row):
    full_path = row['filename']
    class_name = full_path.split('/')[0]
    filename = os.path.basename(full_path)
    return class_name, filename

def move_and_rename_files(row, target_folder):
    class_name, filename = extract_class_and_filename(row)
    source_path = os.path.join(source_folder, class_name, filename)
    target_filename = f"{class_name}_{row.name}.jpg"
    target_path = os.path.join(target_folder, target_filename)

    if os.path.exists(source_path):
        shutil.move(source_path, target_path)
    else:
        print(f"File not found: {source_path}")

train_data = data[data['subset'] == 'train']
train_data.apply(lambda row: move_and_rename_files(row, train_folder), axis=1)

test_data = data[data['subset'] == 'test']
test_data.apply(lambda row: move_and_rename_files(row, test_folder), axis=1)

File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_a-la-place-clichy.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_abricotine.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_affiche-pour-the-century-magazine-napol-on.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_anxi-t-1897.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_bonne-nouvelle.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_encre-l-marquet.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_esclarmonde.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_froideur.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_girl-in-the-garden.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_grafton-gallery-from-les-affiche-illustrees-1897.jpg
File not found: /content/dataset/Art_Nouveau_Modern/eugã¨ne-grasset_harper-s

,0
63998,None
63999,None
64000,None
64001,None
64002,None
...,...
79993,None
79994,None
79995,None
79996,None


In [ ]:
test_folder = "/content/dataset/test"
test_file_count = len([file for file in os.listdir(test_folder) if os.path.isfile(os.path.join(test_folder, file))])

test_file_count

15860

In [ ]:
# clip 전처리 코드 (test)
device = "cuda" if torch.cuda.is_available() else "cpu"
_, preprocess = clip.load("ViT-B/32", device=device)

source_folder = "/content/dataset/test"
processed_folder = "/content/dataset/pt_test"
os.makedirs(processed_folder, exist_ok=True)

print("Preprocessing and saving images...")
for root, _, files in os.walk(source_folder):
    for file in tqdm(files):
        if file.endswith(".jpg") or file.endswith(".png"):
            source_path = os.path.join(root, file)

            relative_path = os.path.relpath(root, source_folder)
            dest_folder = os.path.join(processed_folder, relative_path)
            os.makedirs(dest_folder, exist_ok=True)

            dest_path = os.path.join(dest_folder, file)

            try:
                image = Image.open(source_path).convert("RGB")
                processed_image = preprocess(image)

                torch.save(processed_image, dest_path + ".pt")
            except Exception as e:
                print(f"Error processing file {source_path}: {e}")

100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 90.4MiB/s]


Preprocessing and saving images...


100%|██████████| 15860/15860 [11:19<00:00, 23.34it/s]


In [ ]:
folder_path = "/content/dataset/pt_test"

for root, _, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".jpg.pt"):
            old_path = os.path.join(root, file)
            new_name = file.replace(".jpg.pt", ".pt")
            new_path = os.path.join(root, new_name)

            os.rename(old_path, new_path)
            print(f"Renamed: {old_path} → {new_path}")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Renamed: /content/dataset/pt_test/Rococo_77213.jpg.pt → /content/dataset/pt_test/Rococo_77213.pt
Renamed: /content/dataset/pt_test/Impressionism_69355.jpg.pt → /content/dataset/pt_test/Impressionism_69355.pt
Renamed: /content/dataset/pt_test/Impressionism_69806.jpg.pt → /content/dataset/pt_test/Impressionism_69806.pt
Renamed: /content/dataset/pt_test/Realism_75684.jpg.pt → /content/dataset/pt_test/Realism_75684.pt
Renamed: /content/dataset/pt_test/Baroque_66129.jpg.pt → /content/dataset/pt_test/Baroque_66129.pt
Renamed: /content/dataset/pt_test/Realism_76336.jpg.pt → /content/dataset/pt_test/Realism_76336.pt
Renamed: /content/dataset/pt_test/Romanticism_77480.jpg.pt → /content/dataset/pt_test/Romanticism_77480.pt
Renamed: /content/dataset/pt_test/Naive_Art_Primitivism_72370.jpg.pt → /content/dataset/pt_test/Naive_Art_Primitivism_72370.pt
Renamed: /content/dataset/pt_test/Expressionism_67849.jpg.pt → /content/dataset/pt_test/Expressionism_67849.pt
Ren

In [ ]:
# clip 전처리 (train) random sampling

source_folder = "/content/dataset/train"
destination_folder = "/content/dataset/pt_train"
os.makedirs(destination_folder, exist_ok=True)

random.seed(42)
num_images_per_class = 300

class_images = {}
for file in os.listdir(source_folder):
    if file.endswith(".jpg"):
        class_name = file.split("_")[0]
        if class_name not in class_images:
            class_images[class_name] = []
        class_images[class_name].append(file)

print("Selecting and preprocessing images...")
for class_name, images in tqdm(class_images.items()):
    selected_images = random.sample(images, min(len(images), num_images_per_class))

    for image_name in selected_images:
        source_path = os.path.join(source_folder, image_name)
        dest_name = f"{class_name}_{image_name.replace('.jpg', '.pt')}"
        dest_path = os.path.join(destination_folder, dest_name)

        try:
            image = Image.open(source_path).convert("RGB")
            processed_tensor = preprocess(image)

            torch.save(processed_tensor, dest_path)
        except Exception as e:
            print(f"Error processing {source_path}: {e}")

Selecting and preprocessing images...


100%|██████████| 27/27 [05:15<00:00, 11.70s/it]


In [ ]:
folder_path = "/content/dataset/pt_train"

for file_name in os.listdir(folder_path):
    if file_name.endswith(".pt"):
        parts = file_name.split("_", 2)
        if len(parts) > 2 and parts[0] == parts[1]:
            new_file_name = "_".join(parts[1:])
            os.rename(
                os.path.join(folder_path, file_name),
                os.path.join(folder_path, new_file_name),
            )

##wikiart clip 평가

###clip 제로샷

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

# 클래스 정의
class_names = [
    "Abstract_Expressionism", "Action_painting", "Analytical_Cubism",
    "Art_Nouveau_Modern", "Baroque", "Color_Field_Painting",
    "Contemporary_Realism", "Cubism", "Early_Renaissance",
    "Expressionism", "Fauvism", "High_Renaissance",
    "Impressionism", "Mannerism_Late_Renaissance", "Minimalism",
    "Naive_Art_Primitivism", "New_Realism", "Northern_Renaissance",
    "Pointillism", "Pop_Art", "Post_Impressionism", "Realism",
    "Rococo", "Romanticism", "Symbolism", "Synthetic_Cubism", "Ukiyo_e"
]

display_names = [name.replace("_", " ") for name in class_names]

class_descriptions = {
    "Abstract_Expressionism": "an Abstract Expressionism artwork with random splashes, drips, and spontaneous brushstrokes in bold colors without recognizable shapes",
    "Action_painting": "an Action painting with energetic brushstrokes, paint drips, and splatters showing the artist's physical movements",
    "Analytical_Cubism": "an Analytical Cubism artwork breaking objects into geometric shapes like cubes and triangles, using muted browns and grays",
    "Art_Nouveau_Modern": "an Art Nouveau Modern decorative art with curved lines, swirling patterns, and plant-like shapes in elegant flowing designs",
    "Baroque": "a Baroque painting with strong light-dark contrasts, rich colors, and dramatic compositions with diagonal lines",
    "Color_Field_Painting": "a Color Field Painting with large areas of solid colors painted smoothly without texture or distinct shapes",
    "Contemporary_Realism": "a Contemporary Realism painting with highly detailed photographic-like images, precise shadows and textures",
    "Cubism": "a Cubism artwork showing fragmented images from multiple angles simultaneously, with geometric shapes and overlapping planes",
    "Early_Renaissance": "an Early Renaissance painting with careful attention to perspective, detailed figures, and clear outlines",
    "Expressionism": "an Expressionism artwork with distorted shapes and intense colors that convey strong emotions, often with thick brushstrokes",
    "Fauvism": "a Fauvism painting with unnaturally bright and pure colors, bold brushstrokes and simplified shapes",
    "High_Renaissance": "a High Renaissance artwork with perfectly balanced compositions, realistic human figures and soft, natural lighting",
    "Impressionism": "an Impressionism painting with loose brushstrokes capturing effects of light, with visible dots and dashes of pure color",
    "Mannerism_Late_Renaissance": "a Mannerism Late Renaissance painting with elongated figures in unusual poses, artificial colors and complex compositions",
    "Minimalism": "a Minimalism artwork with extremely simple geometric shapes, solid colors, and clean lines without decoration",
    "Naive_Art_Primitivism": "a Naive Art Primitivism with simple, childlike drawings, flat colors, basic shapes, and lack of perspective",
    "New_Realism": "a New Realism artwork with collages and assemblages of everyday objects and materials",
    "Northern_Renaissance": "a Northern Renaissance painting with extremely detailed surfaces, rich colors, and precise rendering of textures",
    "Pointillism": "a Pointillism artwork made entirely of tiny dots of pure color placed close together",
    "Pop_Art": "a Pop Art with bright colored images of consumer products, celebrities, and comic-style dots and bold outlines",
    "Post_Impressionism": "a Post Impressionism painting with bold colors and thick brushstrokes, emphasis on geometric shapes and patterns",
    "Realism": "a Realism artwork with precise depiction of everyday scenes, natural colors and detailed textures",
    "Rococo": "a Rococo painting with pastel colors, delicate ornate details, and curved shapes in light, airy compositions",
    "Romanticism": "a Romanticism artwork with dramatic landscapes, strong light effects and swirling compositions",
    "Symbolism": "a Symbolism painting with dreamlike images, mysterious figures and unnatural colors",
    "Synthetic_Cubism": "a Synthetic Cubism artwork with collage-like paintings, flat shapes, bright colors, and mixed materials",
    "Ukiyo_e": "a Ukiyo-e Japanese woodblock print with flat colors, bold outlines, and detailed patterns"
}


class TensorDataset(Dataset):
    def __init__(self, tensor_folder):
        self.tensor_folder = tensor_folder
        self.tensor_files = [os.path.join(tensor_folder, file) for file in os.listdir(tensor_folder) if file.endswith(".pt")]

    def __len__(self):
        return len(self.tensor_files)

    def __getitem__(self, idx):
        tensor_path = self.tensor_files[idx]
        tensor = torch.load(tensor_path)
        label = "_".join(os.path.basename(tensor_path).split("_")[:-1])
        return tensor, label

class StyleClassificationEvaluator:
    def __init__(self, device="cuda"):
        self.device = device
        self.model, _ = clip.load("ViT-B/32", device=device)

        # 두 가지 클래스 정의 방식
        self.basic_prompts = {name: f"An artwork in the style of an {name.replace('_', ' ')}"
                            for name in class_names}
        self.detailed_prompts = {k: f"{v}"
                               for k, v in class_descriptions.items()}

    def evaluate_zero_shot(self, dataloader, prompt_type="basic"):
        prompts = self.basic_prompts if prompt_type == "basic" else self.detailed_prompts
        prompt_names = list(prompts.keys())

        text_inputs = clip.tokenize([prompts[name] for name in prompt_names]).to(self.device)
        with torch.no_grad():
            text_features = self.model.encode_text(text_inputs)
            text_features /= text_features.norm(dim=-1, keepdim=True)

        all_preds = []
        all_labels = []
        all_confidences = []

        with torch.no_grad():
            for images, labels in tqdm(dataloader):
                images = images.to(self.device)

                image_features = self.model.encode_image(images)
                image_features /= image_features.norm(dim=-1, keepdim=True)

                similarity = (image_features @ text_features.T)
                confidences = similarity.softmax(dim=-1)
                predictions = similarity.argmax(dim=-1).cpu()

                all_preds.extend(predictions.numpy())
                all_labels.extend([prompt_names.index(label) for label in labels])
                all_confidences.extend(confidences.cpu().numpy())

        return {
            'predictions': np.array(all_preds),
            'labels': np.array(all_labels),
            'confidences': np.array(all_confidences),
            'class_names': prompt_names,
            'display_names': [name.replace("_", " ") for name in prompt_names]
        }

    def analyze_results(self, results, save_prefix):
        preds = results['predictions']
        labels = results['labels']
        confidences = results['confidences']
        display_names = results['display_names']

        # 1. 혼동 행렬
        plt.figure(figsize=(20, 20))
        cm = confusion_matrix(labels, preds)
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        sns.heatmap(cm_normalized, annot=True, fmt='.2f',
                   xticklabels=display_names, yticklabels=display_names)
        plt.title('Normalized Confusion Matrix')
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'{save_prefix}_confusion_matrix.png')
        plt.close()

        # 2. 클래스별 정확도
        class_accuracies = {}
        for i, class_name in enumerate(display_names):
            class_mask = (labels == i)
            class_correct = (preds[class_mask] == labels[class_mask]).sum()
            class_total = class_mask.sum()
            class_accuracies[class_name] = (class_correct / class_total) * 100

        plt.figure(figsize=(15, 8))
        acc_series = pd.Series(class_accuracies)
        acc_series.sort_values().plot(kind='bar')
        plt.title('Classification Accuracy by Class')
        plt.xlabel('Class')
        plt.ylabel('Accuracy (%)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(f'{save_prefix}_class_accuracies.png')
        plt.close()

        # 3. 신뢰도 vs 정확도 산점도
        plt.figure(figsize=(10, 6))
        correct_confidences = confidences[preds == labels].max(axis=1)
        wrong_confidences = confidences[preds != labels].max(axis=1)

        plt.hist(correct_confidences, alpha=0.5, label='Correct Predictions', bins=50)
        plt.hist(wrong_confidences, alpha=0.5, label='Wrong Predictions', bins=50)
        plt.xlabel('Confidence Score')
        plt.ylabel('Count')
        plt.title('Confidence Distribution for Correct vs Wrong Predictions')
        plt.legend()
        plt.tight_layout()
        plt.savefig(f'{save_prefix}_confidence_distribution.png')
        plt.close()

        # 4. 전체 성능 요약
        overall_accuracy = (preds == labels).mean() * 100
        plt.figure(figsize=(8, 6))
        plt.bar(['Overall Accuracy'], [overall_accuracy])
        plt.ylabel('Accuracy (%)')
        plt.title(f'{save_prefix} Classification Performance')
        plt.ylim(0, 100)
        for i, v in enumerate([overall_accuracy]):
            plt.text(i, v + 1, f'{v:.2f}%', ha='center')
        plt.tight_layout()
        plt.savefig(f'{save_prefix}_overall_accuracy.png')
        plt.close()

        return {
            'overall_accuracy': overall_accuracy,
            'class_accuracies': class_accuracies,
            'confusion_matrix': cm_normalized,
        }

    def compare_approaches(self, basic_results, detailed_results):
        # 전체 정확도 계산
        basic_acc = (basic_results['predictions'] == basic_results['labels']).mean() * 100
        detailed_acc = (detailed_results['predictions'] == detailed_results['labels']).mean() * 100

        # 1. 전체 정확도 비교 막대 그래프
        plt.figure(figsize=(10, 6))
        plt.bar(['Basic Prompts', 'Detailed Prompts'], [basic_acc, detailed_acc])
        plt.ylabel('Accuracy (%)')
        plt.title('Overall Accuracy Comparison')
        plt.ylim(0, 100)

        for i, v in enumerate([basic_acc, detailed_acc]):
            plt.text(i, v + 1, f'{v:.2f}%', ha='center')

        plt.tight_layout()
        plt.savefig('accuracy_comparison.png')
        plt.close()

        # 2. 클래스별 정확도 향상도
        class_improvements = {}
        for i, class_name in enumerate(basic_results['display_names']):
            basic_mask = (basic_results['labels'] == i)
            detailed_mask = (detailed_results['labels'] == i)

            basic_class_acc = (basic_results['predictions'][basic_mask] == basic_results['labels'][basic_mask]).mean() * 100
            detailed_class_acc = (detailed_results['predictions'][detailed_mask] == detailed_results['labels'][detailed_mask]).mean() * 100

            class_improvements[class_name] = detailed_class_acc - basic_class_acc

        plt.figure(figsize=(15, 8))
        improvement_series = pd.Series(class_improvements)
        improvement_series.sort_values().plot(kind='bar')
        plt.title('Accuracy Improvement with Detailed Descriptions')
        plt.xlabel('Class')
        plt.ylabel('Accuracy Improvement (%)')
        plt.axhline(y=0, color='r', linestyle='-')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('accuracy_improvements.png')
        plt.close()

        # 3. 산점도: 기본 vs 상세 설명 정확도
        plt.figure(figsize=(10, 10))
        basic_class_accs = {}
        detailed_class_accs = {}

        for i, class_name in enumerate(basic_results['display_names']):
            basic_mask = (basic_results['labels'] == i)
            detailed_mask = (detailed_results['labels'] == i)

            basic_class_accs[class_name] = (basic_results['predictions'][basic_mask] == basic_results['labels'][basic_mask]).mean() * 100
            detailed_class_accs[class_name] = (detailed_results['predictions'][detailed_mask] == detailed_results['labels'][detailed_mask]).mean() * 100

        plt.scatter(list(basic_class_accs.values()), list(detailed_class_accs.values()))
        plt.plot([0, 100], [0, 100], 'r--')
        plt.xlabel('Basic Prompts Accuracy (%)')
        plt.ylabel('Detailed Prompts Accuracy (%)')
        plt.title('Class-wise Accuracy Comparison')

        for class_name in basic_class_accs.keys():
            plt.annotate(class_name,
                        (basic_class_accs[class_name], detailed_class_accs[class_name]),
                        xytext=(5, 5), textcoords='offset points',
                        fontsize=8)

        plt.tight_layout()
        plt.savefig('accuracy_scatter.png')
        plt.close()

        return {
            'overall_improvement': detailed_acc - basic_acc,
            'class_improvements': class_improvements,
            'basic_accuracy': basic_acc,
            'detailed_accuracy': detailed_acc
        }

# 데이터셋 설정 및 평가 실행
tensor_folder = "/content/dataset/pt_test"
dataset = TensorDataset(tensor_folder)
dataloader = DataLoader(dataset, batch_size=1024, shuffle=False)

# 평가기 초기화 및 실행
evaluator = StyleClassificationEvaluator()

# 기본 프롬프트 평가
basic_results = evaluator.evaluate_zero_shot(dataloader, prompt_type="basic")
basic_analysis = evaluator.analyze_results(basic_results, "basic")

# 상세 설명 프롬프트 평가
detailed_results = evaluator.evaluate_zero_shot(dataloader, prompt_type="detailed")
detailed_analysis = evaluator.analyze_results(detailed_results, "detailed")

# 두 접근 방식 비교
comparison = evaluator.compare_approaches(basic_results, detailed_results)

  0%|          | 0/16 [00:00<?, ?it/s]<ipython-input-11-6a5be58322f0>:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(tensor_path)
  0%|          | 0/1

###제로샷 개별 테스트

In [ ]:
#clip 구체적 클래스 설명 제로샷
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

class TensorDataset(Dataset):
    def __init__(self, tensor_folder):
        self.tensor_folder = tensor_folder
        self.tensor_files = [os.path.join(tensor_folder, file) for file in os.listdir(tensor_folder) if file.endswith(".pt")]

    def __len__(self):
        return len(self.tensor_files)

    def __getitem__(self, idx):
        tensor_path = self.tensor_files[idx]
        tensor = torch.load(tensor_path)
        label = "_".join(os.path.basename(tensor_path).split("_")[:-1])
        return tensor, label

tensor_folder = "/content/dataset/pt_test"
class_descriptions = {
    "Abstract_Expressionism": "Abstract Expressionism",
    "Action_painting": "Action painting",
    "Analytical_Cubism": "Analytical Cubism",
    "Art_Nouveau_Modern": "Art Nouveau Modern",
    "Baroque": "Baroque",
    "Color_Field_Painting": "Color Field Painting",
    "Contemporary_Realism": "Contemporary Realism",
    "Cubism": "Cubism",
    "Early_Renaissance": "Early Renaissance",
    "Expressionism": "Expressionism",
    "Fauvism": "Fauvism",
    "High_Renaissance": "High Renaissance",
    "Impressionism": "Impressionism",
    "Mannerism_Late_Renaissance": "Mannerism Late Renaissance",
    "Minimalism": "Minimalism",
    "Naive_Art_Primitivism": "Naive Art Primitivism",
    "New_Realism": "New Realism",
    "Northern_Renaissance": "Northern Renaissance",
    "Pointillism": "Pointillism",
    "Pop_Art": "Pop Art",
    "Post_Impressionism": "Post Impressionism",
    "Realism": "Realism",
    "Rococo": "Rococo",
    "Romanticism": "Romanticism",
    "Symbolism": "Symbolism",
    "Synthetic_Cubism": "Synthetic Cubism",
    "Ukiyo_e": "Ukiyo e"
}

dataset = TensorDataset(tensor_folder)
batch_size = 1024
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

print("Caching text embeddings...")
text_prompts = [f"An artwork in the style of an {desc}" for desc in class_descriptions.values()]
text_inputs = clip.tokenize(text_prompts).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)

correct = 0
total = 0

print("Evaluating zero-shot classification...")
with torch.no_grad():
    for batch in tqdm(dataloader):
        images, labels = batch
        images = images.to(device)

        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        similarity = (image_features @ text_features.T).softmax(dim=-1)
        predictions = similarity.argmax(dim=-1).cpu().numpy()
        total += len(labels)

        for label, pred in zip(labels, predictions):
            predicted_class = list(class_descriptions.keys())[pred]
            if predicted_class == label:
                correct += 1

accuracy = correct / total * 100
print(f"Zero-shot CLIP classification accuracy: {accuracy:.2f}%")

Caching text embeddings...
Evaluating zero-shot classification...


  0%|          | 0/16 [00:00<?, ?it/s]<ipython-input-34-30edfa8d6345>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(tensor_path)
100%|██████████| 16/

Zero-shot CLIP classification accuracy: 23.52%


In [ ]:
#clip 구체적 클래스 설명 제로샷
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

class TensorDataset(Dataset):
    def __init__(self, tensor_folder):
        self.tensor_folder = tensor_folder
        self.tensor_files = [os.path.join(tensor_folder, file) for file in os.listdir(tensor_folder) if file.endswith(".pt")]

    def __len__(self):
        return len(self.tensor_files)

    def __getitem__(self, idx):
        tensor_path = self.tensor_files[idx]
        tensor = torch.load(tensor_path)
        label = "_".join(os.path.basename(tensor_path).split("_")[:-1])
        return tensor, label

tensor_folder = "/content/dataset/pt_test"
class_descriptions = {
    "Abstract_Expressionism": "an Abstract Expressionism artwork with random splashes, drips, and spontaneous brushstrokes in bold colors without recognizable shapes",
    "Action_painting": "an Action painting with energetic brushstrokes, paint drips, and splatters showing the artist's physical movements",
    "Analytical_Cubism": "an Analytical Cubism artwork breaking objects into geometric shapes like cubes and triangles, using muted browns and grays",
    "Art_Nouveau_Modern": "an Art Nouveau Modern decorative art with curved lines, swirling patterns, and plant-like shapes in elegant flowing designs",
    "Baroque": "a Baroque painting with strong light-dark contrasts, rich colors, and dramatic compositions with diagonal lines",
    "Color_Field_Painting": "a Color Field Painting with large areas of solid colors painted smoothly without texture or distinct shapes",
    "Contemporary_Realism": "a Contemporary Realism painting with highly detailed photographic-like images, precise shadows and textures",
    "Cubism": "a Cubism artwork showing fragmented images from multiple angles simultaneously, with geometric shapes and overlapping planes",
    "Early_Renaissance": "an Early Renaissance painting with careful attention to perspective, detailed figures, and clear outlines",
    "Expressionism": "an Expressionism artwork with distorted shapes and intense colors that convey strong emotions, often with thick brushstrokes",
    "Fauvism": "a Fauvism painting with unnaturally bright and pure colors, bold brushstrokes and simplified shapes",
    "High_Renaissance": "a High Renaissance artwork with perfectly balanced compositions, realistic human figures and soft, natural lighting",
    "Impressionism": "an Impressionism painting with loose brushstrokes capturing effects of light, with visible dots and dashes of pure color",
    "Mannerism_Late_Renaissance": "a Mannerism Late Renaissance painting with elongated figures in unusual poses, artificial colors and complex compositions",
    "Minimalism": "a Minimalism artwork with extremely simple geometric shapes, solid colors, and clean lines without decoration",
    "Naive_Art_Primitivism": "a Naive Art Primitivism with simple, childlike drawings, flat colors, basic shapes, and lack of perspective",
    "New_Realism": "a New Realism artwork with collages and assemblages of everyday objects and materials",
    "Northern_Renaissance": "a Northern Renaissance painting with extremely detailed surfaces, rich colors, and precise rendering of textures",
    "Pointillism": "a Pointillism artwork made entirely of tiny dots of pure color placed close together",
    "Pop_Art": "a Pop Art with bright colored images of consumer products, celebrities, and comic-style dots and bold outlines",
    "Post_Impressionism": "a Post Impressionism painting with bold colors and thick brushstrokes, emphasis on geometric shapes and patterns",
    "Realism": "a Realism artwork with precise depiction of everyday scenes, natural colors and detailed textures",
    "Rococo": "a Rococo painting with pastel colors, delicate ornate details, and curved shapes in light, airy compositions",
    "Romanticism": "a Romanticism artwork with dramatic landscapes, strong light effects and swirling compositions",
    "Symbolism": "a Symbolism painting with dreamlike images, mysterious figures and unnatural colors",
    "Synthetic_Cubism": "a Synthetic Cubism artwork with collage-like paintings, flat shapes, bright colors, and mixed materials",
    "Ukiyo_e": "a Ukiyo-e Japanese woodblock print with flat colors, bold outlines, and detailed patterns"
}
dataset = TensorDataset(tensor_folder)
batch_size = 2048
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

print("Caching text embeddings...")
text_prompts = [f"{desc}" for desc in class_descriptions.values()]
text_inputs = clip.tokenize(text_prompts).to(device)
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
    text_features /= text_features.norm(dim=-1, keepdim=True)

correct = 0
total = 0

print("Evaluating zero-shot classification...")
with torch.no_grad():
    for batch in tqdm(dataloader):
        images, labels = batch
        images = images.to(device)

        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        similarity = (image_features @ text_features.T).softmax(dim=-1)
        predictions = similarity.argmax(dim=-1).cpu().numpy()
        total += len(labels)

        for label, pred in zip(labels, predictions):
            predicted_class = list(class_descriptions.keys())[pred]
            if predicted_class == label:
                correct += 1

accuracy = correct / total * 100
print(f"Zero-shot CLIP classification accuracy: {accuracy:.2f}%")

Caching text embeddings...
Evaluating zero-shot classification...


  0%|          | 0/8 [00:00<?, ?it/s]<ipython-input-33-fb65531467b7>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(tensor_path)
100%|██████████| 8/8 

Zero-shot CLIP classification accuracy: 26.70%


##wikiart finetuning

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler
import clip
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os

class FineTuningEvaluator:
    def __init__(self, device="cuda"):
        self.device = device
        self.model, _ = clip.load("ViT-B/32", device=device)
        # 모델을 float32로 변환
        self.model = self.model.float()

        self.basic_prompts = {name: f"An artwork in the style of {name.replace('_', ' ')}"
                            for name in class_names}
        self.detailed_prompts = {k: f"This is a painting of a {v}"
                               for k, v in class_descriptions.items()}

    def train_model(self, train_loader, prompt_type="basic"):
        num_epochs = 25
        learning_rate = 2e-6
        weight_decay = 0.01

        prompts = self.basic_prompts if prompt_type == "basic" else self.detailed_prompts
        prompt_names = list(prompts.keys())

        text_inputs = clip.tokenize([prompts[name] for name in prompt_names]).to(self.device)
        with torch.no_grad():
            text_features = self.model.encode_text(text_inputs)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        optimizer = optim.AdamW(self.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=5, T_mult=2, eta_min=1e-7
        )

        training_history = {
            'train_loss': [],
            'train_acc': [],
            'epoch': []
        }

        for epoch in range(num_epochs):
            self.model.train()
            total_loss = 0
            correct = 0
            total = 0

            for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                images = images.to(self.device)
                label_indices = torch.tensor([prompt_names.index(label) for label in labels]).to(self.device)

                # mixed precision 제거하고 일반 학습으로 변경
                image_features = self.model.encode_image(images)
                image_features = image_features / image_features.norm(dim=-1, keepdim=True)

                similarity = (image_features @ text_features.T)

                criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
                loss = criterion(similarity, label_indices)

                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                optimizer.step()
                optimizer.zero_grad()

                predictions = similarity.argmax(dim=-1)
                correct += (predictions == label_indices).sum().item()
                total += len(labels)
                total_loss += loss.item()

            epoch_loss = total_loss / len(train_loader)
            epoch_acc = (correct / total) * 100

            training_history['train_loss'].append(epoch_loss)
            training_history['train_acc'].append(epoch_acc)
            training_history['epoch'].append(epoch + 1)

            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

            scheduler.step()

        self._plot_training_history(training_history, prompt_type)

        return training_history

    def _plot_training_history(self, history, prompt_type):
        # 손실 그래프
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(history['epoch'], history['train_loss'])
        plt.title(f'Training Loss ({prompt_type} prompts)')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')

        # 정확도 그래프
        plt.subplot(1, 2, 2)
        plt.plot(history['epoch'], history['train_acc'])
        plt.title(f'Training Accuracy ({prompt_type} prompts)')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy (%)')

        plt.tight_layout()
        plt.savefig(f'finetuning_{prompt_type}_training_history.png')
        plt.close()

    def evaluate_model(self, test_loader, prompt_type="basic"):
        self.model.eval()
        prompts = self.basic_prompts if prompt_type == "basic" else self.detailed_prompts
        prompt_names = list(prompts.keys())

        text_inputs = clip.tokenize([prompts[name] for name in prompt_names]).to(self.device)
        with torch.no_grad():
            text_features = self.model.encode_text(text_inputs)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        all_preds = []
        all_labels = []
        all_confidences = []

        with torch.no_grad():
            for images, labels in tqdm(test_loader, desc="Evaluating"):
                images = images.to(self.device)

                image_features = self.model.encode_image(images)
                image_features = image_features / image_features.norm(dim=-1, keepdim=True)

                similarity = (image_features @ text_features.T)
                confidences = similarity.softmax(dim=-1)
                predictions = similarity.argmax(dim=-1).cpu()

                all_preds.extend(predictions.numpy())
                all_labels.extend([prompt_names.index(label) for label in labels])
                all_confidences.extend(confidences.cpu().numpy())

        return {
            'predictions': np.array(all_preds),
            'labels': np.array(all_labels),
            'confidences': np.array(all_confidences),
            'class_names': prompt_names,
            'display_names': [name.replace("_", " ") for name in prompt_names]
        }

# 데이터셋 설정
train_folder = "/content/dataset/pt_train"
test_folder = "/content/dataset/pt_test"

train_dataset = TensorDataset(train_folder)
test_dataset = TensorDataset(test_folder)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

# 기본 프롬프트 학습 및 평가
basic_evaluator = FineTuningEvaluator()
basic_history = basic_evaluator.train_model(train_loader, prompt_type="basic")
basic_results = basic_evaluator.evaluate_model(test_loader, prompt_type="basic")

# Save basic model
torch.save({
    'model_state_dict': basic_evaluator.model.state_dict(),
    'training_history': basic_history
}, 'finetuning_basic_model.pth')

# 상세 설명 프롬프트 학습 및 평가
detailed_evaluator = FineTuningEvaluator()
detailed_history = detailed_evaluator.train_model(train_loader, prompt_type="detailed")
detailed_results = detailed_evaluator.evaluate_model(test_loader, prompt_type="detailed")

# Save detailed model
torch.save({
    'model_state_dict': detailed_evaluator.model.state_dict(),
    'training_history': detailed_history
}, 'finetuning_detailed_model.pth')

# 성능 분석
def analyze_finetuning_results(basic_results, detailed_results):
    basic_acc = (basic_results['predictions'] == basic_results['labels']).mean() * 100
    detailed_acc = (detailed_results['predictions'] == detailed_results['labels']).mean() * 100

    plt.figure(figsize=(10, 6))
    plt.bar(['Basic Prompts', 'Detailed Prompts'], [basic_acc, detailed_acc])
    plt.ylabel('Test Accuracy (%)')
    plt.title('Fine-tuning Test Accuracy Comparison')
    plt.ylim(0, 100)

    for i, v in enumerate([basic_acc, detailed_acc]):
        plt.text(i, v + 1, f'{v:.2f}%', ha='center')

    plt.tight_layout()
    plt.savefig('finetuning_accuracy_comparison.png')
    plt.close()

    return {
        'basic_accuracy': basic_acc,
        'detailed_accuracy': detailed_acc,
        'improvement': detailed_acc - basic_acc
    }

results_comparison = analyze_finetuning_results(basic_results, detailed_results)
print("Fine-tuning Results:")
print(f"Basic Prompts Accuracy: {results_comparison['basic_accuracy']:.2f}%")
print(f"Detailed Prompts Accuracy: {results_comparison['detailed_accuracy']:.2f}%")
print(f"Improvement: {results_comparison['improvement']:.2f}%")

Epoch 1/25:   0%|          | 0/234 [00:00<?, ?it/s]<ipython-input-11-6a5be58322f0>:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(tensor_path)
Epoch 1

Epoch 1/25
Loss: 3.1974, Accuracy: 34.32%


Epoch 2/25: 100%|██████████| 234/234 [00:20<00:00, 11.20it/s]


Epoch 2/25
Loss: 3.1380, Accuracy: 37.94%


Epoch 3/25: 100%|██████████| 234/234 [00:20<00:00, 11.33it/s]


...

Epoch 24/25
Loss: 2.8929, Accuracy: 99.17%


Epoch 25/25: 100%|██████████| 234/234 [00:20<00:00, 11.36it/s]


Epoch 25/25
Loss: 2.8924, Accuracy: 99.51%


Evaluating: 100%|██████████| 16/16 [00:19<00:00,  1.24s/it]


Fine-tuning Results:
Basic Prompts Accuracy: 37.73%
Detailed Prompts Accuracy: 41.03%
Improvement: 3.30%
